# Capstone: Frontend Vision-Enabled UI/UX Agents

**Objective:** Use multi-modal vision models to generate a frontend UI from a design image, and then use a second agent to perform an automated design review.

## Step 1: Setup

**Model Selection:**
This lab requires a vision-capable model. Excellent choices include `gpt-4o`, `gemini-2.5-pro`, or `deepseek-ai/DeepSeek-VL2`.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_vision_completion()`: A specialized function to send an image and a text prompt to a vision model.
- `get_completion()`: To send text-only prompts for the refactoring step.
- `save_artifact()`: To save the generated code and the design review.

In [ ]:
import sys
import os

# Add the project's root directory to the Python path
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_vision_completion, get_image_generation_completion, get_completion, save_artifact, clean_llm_output, load_artifact
from IPython.display import Image, display, Code

image_client, image_model_name, image_api_provider = setup_llm_client(model_name="stabilityai/stable-diffusion-3.5-large")
print(f"✅ Using {image_model_name} for image generation")

# Ensure you select a vision-capable model
vision_client, vision_model_name, vision_api_provider = setup_llm_client(model_name="gpt-4o")
print(f"✅ Using {vision_model_name} for vision tasks")

# Ensure you select a vision-capable model
client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")
print(f"✅ Using {model_name} for text tasks")


if not image_model_name or not vision_model_name:
    print("Could not set up a valid LLM client. Please check your .env file and utils.py configuration.")

2025-10-02 09:49:41,072 ag_aisoftdev.utils INFO LLM Client configured provider=huggingface model=stabilityai/stable-diffusion-3.5-large latency_ms=None artifacts_path=None


✅ Using stabilityai/stable-diffusion-3.5-large for image generation


2025-10-02 09:49:42,668 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=gpt-4o latency_ms=None artifacts_path=None


✅ Using gpt-4o for vision tasks


2025-10-02 09:49:42,992 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=gpt-4o latency_ms=None artifacts_path=None


✅ Using gpt-4o for text tasks


## Step 2: The Design Screenshot

This is the design we want our AI agent to build. A Complex Dashboard component to view goverment contracting data.

In [ ]:
convisoft_prd = load_artifact()

screen_generation_prompt = f""" Your are a senior frontend developer and a UI/UX designer working on building a Complex Dashboard UI using React, MUI / MUIX , and Tailwind CSS.
You have been provided with a PRD to make sure your design includes all the features and requirements: {convisoft_prd}
"""

print(f"Using {image_model_name} for image generation...")
file_path, image_url = get_image_generation_completion(screen_generation_prompt, image_client, image_model_name, image_api_provider)

# Display the generated image if successful
if file_path and image_url:
    print(f"✅ Generated image saved to: {file_path}")
    display(Image(url=image_url))
elif image_url:
    print(f"❌ Image generation failed: {image_url}")
else:
    print("❌ Image generation returned no result")

NameError: name 'get_artifact' is not defined

## Step 3: Generate UI Component using React , MUI / MUIX , Vite, Tailwind CSS

### Generating a Monolithic UI Component

**Task:** Use the vision model to generate a single, self-contained React component that replicates the design from the image.

> **Tip for Vision Prompts:** Be specific about the output format. Telling the model you want 'React' and 'Tailwind CSS' is crucial. The more specific your technical constraints, the better the generated code will be.

**Instructions:**
1.  Create a prompt that asks the vision model to act as an expert frontend developer.
2.  The prompt should instruct the model to analyze the image at the provided URL.
3.  Ask it to generate a single React component using MUI and Tailwind CSS for styling.
4.  The output should be a single block of JSX code.
5.  The output should use MUIX for generating graphs
6.  The output file should be App.tsx and works with vite, axios, and Tailwind CSS.

**Expected Quality:** A single file's worth of React code that, when rendered, visually approximates the login form in the screenshot.

In [ ]:
# Explain the specified image using the vision-capable LLM
image_path = "/Users/labadmin/Documents/AG-AISOFTDEV/artifacts/screens/image_1759247852.png"

explain_image_prompt = f"""
You are an expert frontend developer and UI/UX designer. Analyze the provided image and produce a clear, actionable explanation 
for a developer who will implement the UI.

Please respond in markdown and include the following sections:

1) Summary: One-sentence summary of what this screen is (purpose).
2) Visual Breakdown: Enumerate the visible UI elements (e.g., header, logo, form fields, buttons, icons, images) and approximate positions (top/left/center, grouping).
3) Style Details: Describe colors, font styles (weight/size/line-height if evident), spacing, borders/radii, shadows, and alignment cues.
4) Interaction & Behavior: Identify interactive elements and the likely behaviors (hover/focus states, validation, keyboard affordances).
5) Accessibility Notes: Potential accessibility issues and recommended fixes (labels, contrast, focus order).
6) Implementation Plan (React + Tailwind): Suggest component names and a short mapping of each component to Tailwind utilities or structure (e.g., <LoginForm> -> container: flex, gap-4, ...). Keep this as a concise checklist.

Keep the output developer-focused and actionable.
"""

print("--- Requesting image explanation from vision model ---")
if vision_model_name:
    explanation = get_vision_completion(explain_image_prompt, image_path, vision_client, vision_model_name, vision_api_provider)
    cleaned_explanation = clean_llm_output(explanation, language="markdown")
    display(Code(cleaned_explanation, language="markdown"))
    # Save the explanation for later reference
    save_artifact(cleaned_explanation, "artifacts/image_explanation.md")
else:
    print("Skipping image explanation because no vision model is configured.")

--- Requesting image explanation from vision model ---


# Developer-Focused UI Analysis

## 1) Summary
This screen is an onboarding tool interface for a web application, featuring access points and accompanying policy information.

## 2) Visual Breakdown
- **Header (Top-Left):** Contains a logo icon and the text "Welcome".
- **Main Heading (Center-Left):** "Welcome the Onboarding Tool" with a subheading immediately below.
- **Illustration (Center):** An image next to the main heading.
- **Buttons (Center):** Three buttons labeled "Login 2024", "3 User Accessed", and "Title Amount".
- **Sidebar (Right):** Includes a user/account section ("Hoocut") with a dropdown and additional sections with headings and feature descriptions.
- **Contact Policy Section (Center-Bottom):** Contains cards with icons and text.
- **Footer (Bottom):** Several linked items aligned horizontally.

## 3) Style Details
- **Colors:** Dominant use of purple and soft pastel gradients. Background is a warm gradient of orange to pink.
- **Fonts:** Bold and regular weights. Size and line-height are consistent with modern web typography, with larger sizes for headings.
- **Spacing:** Generous padding and margins between sections. Equal spacing around buttons and cards.
- **Borders/Radii:** Rounded corners on buttons and cards.
- **Shadows:** Subtle shadows on cards to elevate them from the background.
- **Alignment Cues:** Center-aligned main block, right-aligned sidebar.

## 4) Interaction & Behavior
- **Buttons:** Expect hover (lighten color), focus (outline), and active states (darken color).
- **Sidebar Dropdown:** Likely toggles visibility of additional options or settings.
- **Cards:** Possibly clickable, leading to detailed policy pages.
- **Accessibility:** Ensure keyboard focus on all interactive elements.

## 5) Accessibility Notes
- **Contrast:** Verify sufficient contrast between text and background.
- **Labels:** Include aria labels for all interactive elements.
- **Focus Order:** Maintain logical tab order, especially from header to main content and sidebar.
- **Alt Text:** Descriptive text for the illustration and icons in policy cards.

## 6) Implementation Plan (React + Tailwind)
- `<Header>`
  - Container: `flex justify-between items-center p-4 bg-white`
  - Logo: `w-10 h-10`
- `<MainHeading>`
  - Text: `text-4xl font-bold text-black mt-8`
  - Subheading: `text-lg text-gray-600`
- `<Illustration>`
  - Image: `w-1/3`
- `<ButtonGroup>`
  - Button: `bg-purple-600 text-white py-2 px-4 rounded hover:bg-purple-500 focus:outline-none focus:ring`
- `<Sidebar>`
  - Sidebar Container: `fixed right-0 top-0 h-full bg-white shadow-md p-4`
  - Dropdown: `py-2 px-4 bg-gray-100 text-gray-700 rounded`
- `<ContactPolicy>`
  - Card: `flex items-center bg-white shadow-sm rounded p-4 m-2`
  - Icon: `w-12 h-12`
  - Text: `ml-4 text-sm text-gray-700`
- `<Footer>`
  - Footer Container: `flex justify-around items-center p-4 bg-gray-100`

This structured approach should guide implementation while maintaining clarity and focus on both style and functionality.

In [29]:
# TODO: Write a prompt to generate a single React component from the image.
generate_ui_prompt = f"""
You are an expert frontend developer specializing in React and Tailwind CSS.

Your task is to analyze the provided image of a homepage for an onboarding tool with a login/sign-up form that
pops up by clicking on login/signup in the top right corner of the homepage, and write the code for a single, 
self-contained React component that accurately replicates its design and layout. 
Here is the image: {image_path}
Here is some context about the image: {cleaned_explanation}

**Requirements:**
- Use functional components.
- Use Tailwind CSS for all styling. Do not use custom CSS or style tags.
- Make sure the component is accessible, using appropriate HTML tags and attributes.
- The output should be only the raw JSX code for the component.


"""

print("--- Generating Monolithic UI Component ---")
if vision_model_name:
    generated_monolithic_code = get_vision_completion(generate_ui_prompt, image_path, vision_client, vision_model_name, vision_api_provider)
    cleaned_code = clean_llm_output(generated_monolithic_code, language='jsx')
    display(Code(cleaned_code, language='jsx'))
else:
    print("Skipping UI generation because no valid model is configured.")
    cleaned_code = ""

--- Generating Monolithic UI Component ---


import React from 'react';

const OnboardingTool = () => {
  return (
    <div className="min-h-screen bg-gradient-to-r from-orange-400 to-pink-500 p-8">
      <header className="flex justify-between items-center p-4 bg-white shadow-md">
        <div className="flex items-center">
          <div className="w-10 h-10 bg-purple-600 rounded-full"></div>
          <span className="ml-4 text-lg font-bold text-purple-600">Welcome</span>
        </div>
        <button className="bg-purple-600 text-white py-2 px-4 rounded hover:bg-purple-500 focus:outline-none focus:ring">
          Claim User Reward
        </button>
      </header>
      
      <main className="flex justify-between my-8">
        <div className="flex flex-col w-2/3">
          <h1 className="text-4xl font-bold text-white mb-2">Welcome the Onboarding Tool</h1>
          <p className="text-lg text-gray-200 mb-8">Be live a little generated ding would...</p>
          
          <div className="flex space-x-4 mb-8">
            <button className="bg-white text-purple-600 py-2 px-6 rounded shadow hover:bg-gray-100 focus:outline-none focus:ring">
              Login 2024
            </button>
            <button className="bg-purple-600 text-white py-2 px-6 rounded shadow hover:bg-purple-500 focus:outline-none focus:ring">
              3 User Accessed
            </button>
            <button className="bg-white text-purple-600 py-2 px-6 rounded shadow hover:bg-gray-100 focus:outline-none focus:ring">
              Title Amount
            </button>
          </div>

          <section>
            <h2 className="text-2xl font-bold text-white mb-4">Contact Policy</h2>
            <div className="flex space-x-4">
              <div className="flex items-center bg-white shadow-sm rounded p-4">
                <div className="w-12 h-12 bg-gray-200 rounded-full"></div>
                <div className="ml-4">
                  <h3 className="text-sm font-bold text-gray-700">Copying Array Policy</h3>
                  <p className="text-sm text-gray-600">A short description...</p>
                </div>
              </div>
              <div className="flex items-center bg-white shadow-sm rounded p-4">
                <div className="w-12 h-12 bg-gray-200 rounded-full"></div>
                <div className="ml-4">
                  <h3 className="text-sm font-bold text-gray-700">About Coating Printing</h3>
                  <p className="text-sm text-gray-600">A short description...</p>
                </div>
              </div>
            </div>
          </section>
        </div>

        <aside className="fixed right-0 top-0 h-full bg-white shadow-md p-4">
          <div className="flex justify-between items-center mb-4">
            <span className="text-lg font-bold text-gray-700">Hoocut</span>
            <button className="py-2 px-4 bg-gray-100 text-gray-700 rounded">
              ▼
            </button>
          </div>
          <div className="bg-purple-600 p-4 rounded text-white mb-4">
            <h2 className="font-bold">All Rising Password</h2>
          </div>
          <div>
            <h3 className="font-bold text-gray-700">Features</h3>
            <p className="text-sm text-gray-600">Short description...</p>
          </div>
        </aside>
      </main>

      <footer className="flex justify-around items-center p-4 bg-gray-100 mt-8">
        <span className="text-gray-600">Terms of Service</span>
        <span className="text-gray-600">Privacy Policy</span>
        <span className="text-gray-600">Contact</span>
      </footer>
    </div>
  );
};

export default OnboardingTool;

### Challenge 2 (Intermediate): Refactoring into Reusable Components

**Task:** A single, large component is not good practice. Now, prompt the LLM to refactor the monolithic code it just generated into smaller, reusable sub-components.

**Instructions:**
1.  Create a new prompt.
2.  Provide the monolithic JSX code from the previous step as context.
3.  Instruct the LLM to act as a senior frontend developer who champions clean code.
4.  Ask it to refactor the code by creating smaller, reusable components (e.g., `<StyledButton>`, `<InputWithIcon>`).
5.  The final output should be the complete code with the new, smaller components defined and used within the main `Login` component.

**Expected Quality:** A well-structured React file that demonstrates the component-based architecture, which is a fundamental best practice in modern frontend development.

In [35]:
# TODO: Write a prompt to refactor the monolithic code into smaller components.
refactor_ui_prompt = f"""
# Here is the code for a single React component that implements a login form UI on a homepage for an onboarding tool with a login/sign-up form that
# pops up by clicking on login/signup in the top right corner of the homepage.
Your task is to refactor this code into smaller, reusable React components using Tailwind CSS for styling.
Refactor the code by creating smaller, reusable componets like <StyledButton>, <InputField>, <Header>, <Footer>, <LoginForm>, <NavigationBar>, etc.
The final outpout should be complete code with the new , amller compeneds defined and used within the main Login component.
Remember to provide the code generated from the context of the image: {cleaned_explanation}

Output only the raw, complete, refactored JSX code.
"""

print("--- Refactoring UI into Components ---")
if cleaned_code:
    refactored_code = get_completion(refactor_ui_prompt, client, model_name, api_provider)
    cleaned_refactored_code = clean_llm_output(refactored_code, language='jsx')
    display(Code(cleaned_refactored_code, language='jsx'))
else:
    print("Skipping refactoring because monolithic code was not generated.")
    cleaned_refactored_code = ""

--- Refactoring UI into Components ---


import React, { useState } from 'react';

const StyledButton = ({ label, onClick }) => (
  <button
    onClick={onClick}
    className="bg-purple-600 text-white py-2 px-4 rounded hover:bg-purple-500 focus:outline-none focus:ring"
  >
    {label}
  </button>
);

const InputField = ({ type, placeholder, value, onChange }) => (
  <input
    type={type}
    placeholder={placeholder}
    value={value}
    onChange={onChange}
    className="border p-2 rounded w-full mb-4"
  />
);

const Header = () => (
  <header className="flex justify-between items-center p-4 bg-white">
    <div className="flex items-center">
      <img src="/logo.png" alt="Logo" className="w-10 h-10" />
      <span className="text-xl font-bold">Welcome</span>
    </div>
    <div>
      <StyledButton label="Login/Signup" onClick={() => console.log('Open Modal')} />
    </div>
  </header>
);

const MainHeading = () => (
  <div className="text-center mt-8">
    <h1 className="text-4xl font-bold text-black">Welcome to the Onboarding Tool</h1>
    <p className="text-lg text-gray-600">An easy way to get started</p>
  </div>
);

const Illustration = () => (
  <div className="flex justify-center mt-8">
    <img src="/illustration.png" alt="Illustration" className="w-1/3" />
  </div>
);

const ButtonGroup = () => (
  <div className="flex justify-center space-x-4 mt-8">
    <StyledButton label="Login 2024" onClick={() => console.log('Login 2024')} />
    <StyledButton label="3 User Accessed" onClick={() => console.log('3 User Accessed')} />
    <StyledButton label="Title Amount" onClick={() => console.log('Title Amount')} />
  </div>
);

const Sidebar = () => (
  <aside className="fixed right-0 top-0 h-full bg-white shadow-md p-4">
    <div className="py-2 px-4 bg-gray-100 text-gray-700 rounded mb-4">Hoocut</div>
    <div className="py-2 px-4 bg-gray-100 text-gray-700 rounded">Additional Options</div>
  </aside>
);

const ContactPolicy = () => (
  <div className="flex justify-center mt-8">
    <div className="flex items-center bg-white shadow-sm rounded p-4 m-2">
      <img src="/icon.png" alt="Policy Icon" className="w-12 h-12" />
      <p className="ml-4 text-sm text-gray-700">Policy Information</p>
    </div>
  </div>
);

const Footer = () => (
  <footer className="flex justify-around items-center p-4 bg-gray-100">
    <a href="#" className="text-gray-700">Link 1</a>
    <a href="#" className="text-gray-700">Link 2</a>
    <a href="#" className="text-gray-700">Link 3</a>
  </footer>
);

const LoginForm = ({ onClose }) => {
  const [email, setEmail] = useState('');
  const [password, setPassword] = useState('');

  const handleSubmit = (e) => {
    e.preventDefault();
    console.log('Email:', email, 'Password:', password);
  };

  return (
    <div className="fixed inset-0 bg-black bg-opacity-50 flex justify-center items-center">
      <div className="bg-white p-6 rounded shadow-md w-1/3">
        <h2 className="text-2xl font-bold mb-4">Login</h2>
        <form onSubmit={handleSubmit}>
          <InputField
            type="email"
            placeholder="Email"
            value={email}
            onChange={(e) => setEmail(e.target.value)}
          />
          <InputField
            type="password"
            placeholder="Password"
            value={password}
            onChange={(e) => setPassword(e.target.value)}
          />
          <div className="flex justify-end">
            <StyledButton label="Submit" onClick={handleSubmit} />
          </div>
        </form>
        <button onClick={onClose} className="mt-4 text-gray-500">Close</button>
      </div>
    </div>
  );
};

const Login = () => {
  const [showLoginForm, setShowLoginForm] = useState(false);

  return (
    <div className="min-h-screen bg-gradient-to-r from-orange-400 to-pink-400">
      <Header />
      <MainHeading />
      <Illustration />
      <ButtonGroup />
      <Sidebar />
      <ContactPolicy />
      <Footer />
      {showLoginForm && <LoginForm onClose={() => setShowLoginForm(fa

### Challenge 3 (Advanced): The AI UI/UX Critic Agent

**Task:** Create a new "UI/UX Critic" agent. This agent will be given both the original design image and the generated code, and its job is to perform an automated design review.

**Instructions:**
1.  Create a final, complex prompt for a new agent.
2.  The prompt should instruct the agent to act as a meticulous UI/UX designer.
3.  Provide the agent with two pieces of context: the URL of the original design image and the final, refactored React code.
4.  The agent's task is to compare the code's likely rendered output to the design image and list any visual inconsistencies in spacing, font size, color, or layout.

**Expected Quality:** A critical design review in markdown format. This demonstrates a powerful AI-on-AI workflow, where one AI generates work and another AI validates it, automating a time-consuming QA step.

In [37]:
# TODO: Write a prompt for the UI/UX Critic agent.
critic_agent_prompt = f"""
You are a meticulous UI/UX designer with a sharp eye for detail. Your task is to perform a design review.

You will be given two things:
1.  An image of the original design mockup.
2.  The React code that was generated to implement that design.

Your job is to compare the likely visual output of the code to the original design image. Identify and list any visual inconsistencies, no matter how small. Pay close attention to:
- Spacing and padding
- Font sizes and weights
- Color hex codes
- Border radius and shadows
- Overall layout and alignment

**Generated Code:**
```jsx
{cleaned_refactored_code}
```

Provide your feedback as a markdown-formatted list.
"""

print("--- Invoking UI/UX Critic Agent ---")
if cleaned_refactored_code:
    design_review = get_vision_completion(critic_agent_prompt, image_path, vision_client, vision_model_name, vision_api_provider)
    display(Code(design_review, language='markdown'))
    save_artifact(design_review, "artifacts/design_review.md")
else:
    print("Skipping critic agent because refactored code is not available.")

--- Invoking UI/UX Critic Agent ---


Based on comparing the React code with the provided design mockup image, here is a list of visual inconsistencies and feedback:

1. **Header:**
   - **Font Weight:** The mockup uses a bolder font for "Welcome" than is specified in the code (`font-bold` is correct, but it may require extra weight).
   - **Spacing:** The space between the logo and "Welcome" text seems larger in the mockup.
   - **Color and Style:** The "Claim User Reward" button in the mockup has a distinctive background color and text style that isn't captured in the code.

2. **Main Heading:**
   - **Font Style:** The title font in the mockup appears to be a different style or weight than provided (`text-4xl font-bold` might not match exactly).
   - **Subtext Style:** The subtext is more faded in the mockup image; it may require decreased opacity or a lighter gray (`text-gray-600`).

3. **Illustration:**
   - **Alignment and Size:** The illustration's alignment and size must be checked to match the mockup, where it seems more centered and larger.

4. **Button Group:**
   - **Button Style:** The buttons in the mockup have a more pronounced border and shadow compared to the code. The color and text style also differ (e.g., the purple background isn't reflected in the code).

5. **Sidebar:**
   - **Background Color:** The sidebar in the mockup has a unique background, possibly a gradient that is not reflected in the code.
   - **Component Design:** Components like "Features" and other options have distinct styling and layout in the mockup, which the code must match.

6. **Contact Policy:**
   - **Element Spacing:** The spacing between the policy cards and alignment appears different in the mockup.
   - **Card Style:** The card border and shadow specifics are missing in code (`shadow-sm` may need adjustment).

7. **Footer:**
   - **Link Style:** The footer links in the mockup appear to have different styling, particularly the hover effects.
  
8. **General Design:**
   - **Color Scheme:** Verify all color codes to match the specified hex values in the mockup, as the gradient and some elements may use custom colors.
   - **Border Radius and Shadows:** Check that all borders and shadows specifications align with the mockup, particularly prominent in buttons and cards.
   - **Overall Layout:** Ensure that the vertical spacing and alignment across components correspond to the mockup, particularly the balance between sections.

Consider updating the CSS classes or adding custom styles to closely align the code's output with the design mockup.

## Lab Conclusion

Fantastic! You have completed a full, end-to-end frontend development workflow using multiple AI agents. You used a vision-powered agent to generate code from a design, a refactoring agent to improve the code's structure, and a critic agent to perform an automated design review. This powerful combination of skills can dramatically accelerate the process of turning visual ideas into functional user interfaces.

> **Key Takeaway:** The workflow of **Generate -> Refactor -> Critique** is a powerful AI-assisted development pattern. Using specialized agents for each step allows you to rapidly create a first draft, improve its quality, and then automatically check it for correctness, significantly speeding up the iteration cycle.